https://www.nltk.org/api/nltk.tokenize.treebank.html

In [ ]:
# from gensim.models import KeyedVectors

# # Replace 'path/to/word2vec/model.bin' with the path to your pretrained Word2Vec model
# model_path = 'path/to/word2vec/model.bin'
# word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

# # Access the vector for a specific word
# vector_for_word = word2vec_model['example']


In [6]:
import re
import pandas as pd
import nltk
from nltk.tokenize import TreebankWordTokenizer

In [128]:
class SentenceToken:
  '''
    SentenceToken

  '''
  def __init__(self, sentence, aspect_type=None, aspects=None, sentence_id=None):
    
    if sentence_id is not None:
      print(sentence_id)
    self.sentence_id = sentence_id
    self.sentence = sentence
    self.aspect_bio_tags = None
    self.unified_aspect_bio_tags = None
    self.token_span = None
    self.space_pre_token = None

    # Tokenize sentence
    self.__tokenize_sentence(sentence)

    if aspect_type == 'dict':
      self.set_aspect_tagging_from_dict(aspects)
    elif aspect_type == 'bio':
      self.set_aspect_bio_tags(aspects)
    elif aspect_type == 'unified bio':
      self.set_aspect_unified_bio_tags(aspects)
  
  def __tokenize_sentence(self, sentence):
    self.sentence = sentence
    
    token_span = list(TreebankWordTokenizer().span_tokenize(sentence))

    new_token_span = []
    
    # for k in token_span:
    #   token_start = k[0]
    #   token_end = k[1]

    #   token = self.sentence[token_start:token_end]
      # sub_token_split_index = re.finditer(r'([^\w,\d])', token)
      
      # sub_token_start = token_start 

      # for m in sub_token_split_index:
      #   if m.start() != 0:
      #     sub_token_end = token_start + m.start()
      #     new_token_span.append((sub_token_start, sub_token_end))
      #     new_token_span.append((sub_token_end, sub_token_end+1))
      #     sub_token_start = sub_token_end + 1

      # new_token_span.append((sub_token_start, token_end))

    for k in token_span:
      token_start = k[0]
      token_end = k[1]

      token = sentence[token_start:token_end]
      sub_tokens = re.split(r'([^\w,\d])', token)
      
      sub_token_start = token_start
      for sub_token in sub_tokens:
        if len(sub_token) != 0:
          sub_token_end = sub_token_start + len(sub_token)
          new_token_span.append((sub_token_start, sub_token_end))
          sub_token_start = sub_token_end
  
    self.token_span = new_token_span
    self.space_pre_token = [True if sentence[k[0]-1:k[0]] == ' ' else False for i,k in enumerate(self.token_span)]


  def set_aspect_tagging_from_dict(self, aspects):
    polarity_map = {'positive':'POS'
              ,'negative': 'NEG'
              ,'conflict': 'CON'
              ,'neutral': 'NEU'}
    
    bio_tags = ['O'] * len(self.token_span)
    unified_bio_tags = bio_tags

    for x in aspects:
      if x['term'] != '':
        aspect_from = int(x['from'])
        aspect_to = int(x['to'])
        polarity = '-' + polarity_map[x['polarity']]
        aspect_token_ids =  [i for i, v in enumerate(self.token_span) if (v[0] >= aspect_from) & (v[1] <= aspect_to)]
        aspect_from_index = min(aspect_token_ids)
        aspect_to_index = max(aspect_token_ids)
        aspect_length = aspect_to_index - aspect_from_index
        bio_tags = bio_tags[:aspect_from_index] + ['B'] + ['I'] * (aspect_length) + bio_tags[aspect_to_index+1:]
        unified_bio_tags = unified_bio_tags[:aspect_from_index] + ['B' + polarity] + ['I'+ polarity] * (aspect_length) + unified_bio_tags[aspect_to_index+1:]
    
    self.set_aspect_bio_tags(bio_tags)
    self.set_aspect_unified_bio_tags(unified_bio_tags)

  def rebuild_sentence_from_token(self):
    return ''.join([(' ' if self.space_pre_token[i] else '') + self.sentence[k[0]:k[1]] for i, k in enumerate(self.token_span)])
  
  def get_sentence_token_with_aspect_bio_tag(self, unified_bio_tag=False):
    if (unified_bio_tag == False) & (self.aspect_bio_tags is None):
      raise Exception('No BIO tags provided. Use "SentenceToken.set_aspect_bio_tags()" method to add bio_tags')
    elif (unified_bio_tag == True) & (self.aspect_unified_bio_tags is None):
      raise Exception('No Unified BIO tags provided. Use "SentenceToken.set_aspect_unified_bio_tags()" method to add unified_bio_tags')
    else:
      return [(self.sentence[k[0]:k[1]], self.aspect_unified_bio_tags[i] if unified_bio_tag else self.aspect_bio_tags[i]) for i, k in enumerate(self.token_span)]

  def set_aspect_bio_tags(self, aspect_bio_tags):
    self.aspect_bio_tags = aspect_bio_tags
    self.aspect_unified_bio_tags = aspect_bio_tags

  def set_aspect_unified_bio_tags(self, aspect_unified_bio_tags):
    self.aspect_unified_bio_tags = aspect_unified_bio_tags
    self.set_aspect_bio_tags([k[0:1] for k in aspect_unified_bio_tags])

  def check_rebuild_sentence_from_token(self):
    return self.sentence == self.rebuild_sentence_from_token()
  
  def get_tokens(self):
    return [self.sentence[k[0]:k[1]] for k in self.token_span]

  def check_rebuild_aspect_terms(self, aspect_dict):
    aspect_dict = sorted(aspect_dict, key=lambda d: int(d['from']))
    aspect_input = ', '.join([k['term'] for k in aspect_dict])
    aspect_computed = ''.join([(', ' if k == 'B' else '') + self.sentence[self.token_span[i][0]:self.token_span[i][1]] for i,k in enumerate(self.aspect_bio_tags) if k in ['B','I'] ])[2:]
    
    return (aspect_input == aspect_computed, aspect_input, aspect_computed)

  def __str__(self):
    return self.rebuild_sentence_from_token()


In [129]:
# def encode_bio( sentence, aspects):


#   sentence_token = SentenceToken(sentence, 'dict', aspects)

#   return sentence_token.check_rebuild_sentence_from_token(), sentence_token

# check, token = encode_bio(df_train['text'][612], df_train['aspects'][612])
# print(token)
# print(token.get_sentence_token_with_aspect_bio_tag())
# print(token.get_sentence_token_with_aspect_bio_tag(True))
# token.check_rebuild_aspect_terms(df_train['aspects'][612])

In [131]:
df_train = pd.read_json('data/laptop/train.json')
# df_train.set_index('id', inplace=True).reset_index()
print('df_train shape: ', df_train.shape)

df_val = pd.read_json('data/laptop/validate.json') # This will only be used for the very last step to evaluate how well the model is, but is input now for validating the BIO tagging to ensure the function works properly
# df_val.set_index('id', inplace=True).reset_index()
print('df_val shape: ', df_val.shape)

# First, I will need to drop some duplicated data in our training dataset, as identified in the EDA process.
df_train.drop_duplicates(subset='text', inplace=True)

# We have removed 12 duplicated records in our training dataset
print(df_train.shape)

# df_train = df_train[~df_train['id'].isin([1650,1050])]

df_train shape:  (3048, 3)
df_val shape:  (800, 3)
(3036, 3)


In [133]:
df_train['sentence_token'] = df_train.apply(lambda x: SentenceToken(x['text'], 'dict', x['aspects']), axis=1)
df_val['sentence_token'] = df_val.apply(lambda x: SentenceToken(x['text'], 'dict', x['aspects']), axis=1)

In [134]:
t = df_train[df_train['id']==1793]
# s = SentenceToken(t['text'].values[0], 'dict', t['aspects'].values[0])
s = SentenceToken(t['text'].values[0])
print(s.sentence)
print(s.get_tokens())
print(s.token_span)

Without a doubt, the *design* of this laptop is fantastic.
['Without', 'a', 'doubt', ',', 'the', '*', 'design', '*', 'of', 'this', 'laptop', 'is', 'fantastic', '.']
[(0, 7), (8, 9), (10, 15), (15, 16), (17, 20), (21, 22), (22, 28), (28, 29), (30, 32), (33, 37), (38, 44), (45, 47), (48, 57), (57, 58)]


In [137]:
df_train[['aspect_check', 'aspect ']] = df_train['sentence_token'].apply(lambda x: x.check_rebuild_aspect_terms())

KeyError: 'sentence_token'

In [88]:
# print(df_train['sentence_token'][15])
# print(df_train['sentence_token'][15].get_sentence_token_with_aspect_bio_tag())

In [89]:
s = df_train.loc[df_train['id']==3017]
print(s['text'].values[0])
print(s['aspects'].values[0])


The improvements to the OS have been relatively gradual, but substantive.
[{'term': 'OS', 'polarity': 'positive', 'from': '24', 'to': '26'}]


In [127]:
new_token_span = []
sentence = 'Without a doubt, the **design* of this laptop is fantastic.'
token_span  = [(21,30)]
for k in token_span:
  token_start = k[0]
  token_end = k[1]

  token = sentence[token_start:token_end]
  sub_tokens = re.split(r'([^\w,\d])', token)
  
  sub_token_start = token_start
  for sub_token in sub_tokens:
    if len(sub_token) != 0:
      sub_token_end = sub_token_start + len(sub_token)
      new_token_span.append((sub_token_start, sub_token_end))
      sub_token_start = sub_token_end

  # new_token_span.append((sub_token_start, token_end))

print(new_token_span)
print([sentence[k[0]:k[1]] for k in new_token_span])

[(21, 22), (22, 23), (23, 29), (29, 30)]
['*', '*', 'design', '*']


In [39]:
t = s['text'].values[0]
t[24:25]

'O'

In [124]:
re.split(r'([^\w,\d])', '..ajd')

['', '.', '', '.', 'ajd']